# Tutorial #2: Cantera and User-Defined Functions

__Motivation:__

Now is the point in this course where we will introduce and use the Cantera software package to perform detailed
 kinetic simulations.
Some of you may have previously used Cantera in Thermodynamics to access thermodynamic data and view equilibrium
 chemistry.
Regardless of whether you have had previous exposure, here we're going to introduce you to the chemical-kinetic
 capabilities of Cantera and guide you through using them.

## 1. User-Defined Functions

Prior to launching into Cantera, we first take a moment to introduce user-defined functions.
Defining functions allows for modularizing and reusing code, which greatly simplifies the implementation of complex
 computational tasks.

The basic form for defining a function in Python is as follows:
```
def <function_name>(<args*>, <kwargs**>):
    code...
    ...
    ...
    return <return variable(s)>
```

In this form:
* `def` is the keyword that tells Python we are defining a function
* `<function_name>` is the name we will use to reference our new function
* `<args*>` is a list of required arguments (variables) that are:
  * required by the function
  * provided to the function in a pre-determined order
* `<kwargs*>` is a list of keyword arguments that are:
  * optional whether you provide them when running the function
  * specified with a default value that is used if no other value is provided
* `:` ends the function definition line
* whitespace: code within a function must be consistently padded by whitespace
  * Python interprets encountering non-tabbed code as having reached the end of the function
* `return` keyword passes the argument(s) listed after it back to the calling code
  * If more than one value is listed, the values are grouped and returned as a tuple

Let's look at some example function definitions, starting with the area of a triangle

In [ ]:
def triangle_area(base, height):
    return 0.5 * base * height

triangle_area(3, 2)

That wasn't so hard!

While this example is pretty trivial, it let's us see how we might use it to implement more advanced functionality, like
 input validation.

In [ ]:
def triangle_area(base, height):
    if base < 0 or height < 0:
        raise Exception('Base and height of a triangle must be non-negative')
    return 0.5 * base * height

print(triangle_area(3, 2))
# print(triangle_area(-3, 2))

We know that the dimensions of a triangle have to be positive, so we can check for that and `raise` a custom `Exception`
 message if a negative value is encountered.

To illustrate the use of keyword arguments, imagine we want to compute points along a line of the form `y = mx + b`,
 and let's assume that `b` is _usually_ zero.
We want to make our function generalizable (i.e. be _able_ to specify other values of `b`), but we don't want to have to
 enter `b` in all the cases where it is, in fact, zero.

In [ ]:
def y(m, x, b=0):
    return m * x + b

print(y(2, 3))
print(y(0, 2))
print(y(0, 2, b=1.))

In the first two cases, we don't specify a value for `b`, and the default of zero is used.
In the final example, the value of `b` is overwritten by providing a keyword argument.

## 2. Cantera

We are finally here, ready to introduce Cantera for Python.
The material covered in this tutorial is not meant to be comprehensive, but should cover what you need to know to do the
 problem set.
We additionally hope this material will allow you to get you feet wet enough to be able to start making sense of the
 documentation, describing the full functionality of Cantera, which can be found here:
[Cantera Documentation](https://cantera.org/documentation/docs-2.4/sphinx/html/cython/index.html)

For consistency, we always import Cantera using the name `ct`.
We will also go ahead and import some additional useful packages right away.

In [ ]:
import cantera as ct
import numpy as np
import matplotlib.pyplot as plt
import os

### 2a. Initialize a solution object

For this example, we will use the "Stanford Mechanism v1.2".
This mechanism is not built-in to Cantera (the way 'GRI Mech 3.0' is), so we must get the file path to the mechanism
 '.cti' file stored elsewhere in the repository.

In [ ]:
mech_file = os.path.abspath('../mechanisms/stanford-mech_v12.cti')
mech_file

We can now pass this filepath as an argument to the `ct.Solution` class to create an instance of the class based on this
 mechanism.

Placing empty parenthesis `()` after the Solution instance name produces an output of it's state.

In [ ]:
gas = ct.Solution(mech_file)
gas()

### 2b. Accessing and assigning state variables

Properties, accessed with dot notation but without `()` or arguments, are used to access state properies of `gas`

In [ ]:
print(f'T = {gas.T:3.1f} K')
print(f'P = {gas.P:3.0f} Pa')
print(f'rho = {gas.density_mass:5.4f} kg/m^3')
print(f'TPX = {gas.TPX}')

We can also set properties through assignment.
We cannot assign temperature or pressure one at a time, as it would result in ambiguity as to what constraint to
 enforce (constant volume, constant enthalpy, constant entropy, etc.).
However, we can assign pairs of state variables, wherein we can maintain a value as constant.

In [ ]:
gas.TP = 1200, gas.P
print(f'T = {gas.T:3.1f} K')
print(f'P = {gas.P:3.0f} Pa')

gas.TP = gas.T, 1e6
print(f'T = {gas.T:3.1f} K')
print(f'P = {gas.P:3.0f} Pa')

### 2c. Setting and retrieving mole fractions

During simulations, we will often want to monitor the mole fraction of a species or species.

If we use the `X` property to get the mole fractions...

In [ ]:
gas.X

We get an array of values, corresponding, in order, to the species in the mechanism:

In [ ]:
gas.species_names

While this might work okay in some instances, it is a bit tedious to figure out which mole fraction corresponds to each
 species.
Not that we can't do it, e.g.

In [ ]:
for sp, x in zip(gas.species_names, gas.X):
    print(f'{sp:5s}: {x}')

Depending what we are doing with it, it may be easier to have the mole fractions in a dictionary.
`mole_fraction_dict` takes an optional threshold argument (default 0), and only returns species with mole fractions
 above the threshold.
Setting the threshold negative returns all species.

In [ ]:
gas.mole_fraction_dict()

In [ ]:
gas.mole_fraction_dict(-1)

It is almost always easier to set mole fractions using a dictionary instead of an array.
If mole fractions do not sum to one, they are normalized automatically.

In [ ]:
xs = {'N2':.79, 'O2':.21, 'H2':.21/0.5}  # stoichiometric H2 in air
gas.X = xs

gas.mole_fraction_dict()

Alternatively, we can separately define fuel and oxidizer compositions and set the mole fractions by equivalence ratio:
This can save us work having to recalculate component mole fractions.

In [ ]:
fuel = {'H2':1}
ox = {'AR':.95, 'O2':.05}
gas.set_equivalence_ratio(1, fuel, ox)

gas.mole_fraction_dict()

### 2d. Setting up reactor and network objects

Now that we have a gas object, we can set up our reactor and network:

In [ ]:
r1 = ct.ConstPressureReactor(gas)
rnet = ct.ReactorNet((r1,))  # r1 must be passed within a tuple to the reactor network

rnet.time

We see in the above that we now have a reactor at time zero.

If we step the reactor:

In [ ]:
rnet.step()

print(f'Time: {rnet.time:6.5e} s')
gas.mole_fraction_dict()

Repeatedly run the cell above and observe how, at every step:
* the reactor time increases
* the mole fractions change

It is tedious (and pointless) to do this manually, so let's wrap the step function in a loop to step it forward until a
 certain time is reached.
We'll add a counter as well to see how many steps it takes.

In [ ]:
t_stop = 1e-3  # 1 ms
count = 0

while rnet.time < t_stop:
    rnet.step()
    count += 1

print(f'{count} TIme Steps Taken')
print(f'Time: {rnet.time:6.5e} s')
gas.mole_fraction_dict()

### 2e. Collecting simulation results

We've now seen how to run a simulation, but we have not, so far, actually saved any of our simulation results! Oh no!

Let's reset our simulation and see what we can do to record its change through time.

We'll find we need to reset our simulations a lot between runs.
To save time, let's use a function to do this for us.

In the cell below, we first define some default parameters for our simulation.
Then, we define a function to reset the `Solution` object to the initial state, allowing our default parameters to be
 overwritten.
We'll also create a new reactor and network in the function, and return them.

In [ ]:
T0 = 1200  # K
P0 = 101325  # Pa (1 atm)
phi = 1.
fuel = {'H2':1.}
oxidizer = {'AR':.79, 'O2':.21}  # "airgon"

def setup_simulation(sln, T=T0, P=P0, phi=phi, f=fuel, ox=oxidizer):
    sln.set_equivalence_ratio(phi=phi, fuel=f, oxidizer=ox)
    sln.TP = T, P
    r1 = ct.ConstPressureReactor(sln)
    rnet = ct.ReactorNet([r1])
    return r1, rnet

And that's it!
We don't have to return the solution object (but we could if we wanted).
When passed to a function, only a _reference_ to the `Solution` object is passed, not a copy of the object.
Therefore, anything done to the object within the function also affects the object outside the function.

__Note:__ This is an important subtlety of Python and can lead to unexpected behavior if not understood.
For the sake of this class, we have tried to structure things such that this behavior (hopefully) won't cause you
 trouble.

We do return the __Reactor__ and __ReactorNet__ objects, as those are newly created within the function.
Their _scope_ only extends until the function concludes, unless their references are returned.

Let's test it:

In [ ]:
r1, rnet = setup_simulation(gas)
gas()

There we have it, we're back where we want to be, with unreacted fuel-air mixture.

If we look at the reactor network:

In [ ]:
rnet.time

It's back at time zero, since we created a brand-new object. 
Before we start stepping our simulation again, let's figure out how to store our results.

A simple method is to store the results for each variable of interest in `list`.
Let's see what that might look like:

In [ ]:
# We can define empty lists for each variable of interest
time = []
T = []
H2 = []
O2 = []
OH = []
H2O = []

In [ ]:
# We can then run the simulation, appending items to the lists for each step
t_stop = 1e-3  # 1 ms

while rnet.time < t_stop:
    rnet.step()
    time += [rnet.time]
    T += [gas.T]
    xs = gas.mole_fraction_dict(-1)
    H2 += [xs['H2']]
    O2 += [xs['O2']]
    OH += [xs['OH']]
    H2O += [xs['H2O']]

We call the `mole_fraction_dict` method with the argument `-1` to make sure that all species in the mechanism have
 entries in the dictionary.
If we didn't do this, species with mole fractions of zero would not appear in the dictionary and would raise errors
 when accessed.

And now we can plot our results:

In [ ]:
plt.figure(figsize=(8,4))
plt.plot(time, T)
plt.xlabel('Time (s)')
plt.ylabel('Temperature (K)')

plt.figure(figsize=(8,4))
for sp, x in zip(['H2', 'O2', 'OH', 'H2O'], [H2, O2, OH, H2O]):
    plt.loglog(time, x, label=sp)
plt.xlabel('Time (s)')
plt.ylabel('Mole Fraction (-)')
plt.xlim((1e-5, 1e-3))
plt.ylim((1e-6, 1))
plt.legend()

And there we have it - our first kinetic simulation time-series data!

__An aside on measuring computational efficiency__

Another option getting mole fractions would be to call `mole_fraction_dict` with no argument and use the the `get`
 method to access dictionary keys, passing the second argument `0` as a default if the key is not found
 (i.e. species doesn't appear in the `mole_fraction_dict`).

Since getting mole fractions is something that will be done many times during simulations, we might want to make sure
 we're doing this as efficiently as possible.
We can set up cells with the different access methods, using the "cell magic" `&&timeit` in a cell to tell Jupyter to
 compute how long a cell takes to run.

In [ ]:
%%timeit

# return all species in the dictionary, then access the dictionary through dictionary indexing
gas.mole_fraction_dict(-1)['H2']

In [ ]:
%%timeit

# return only species with non-zero mole fractions, then access the dictionary with the 'get' method
gas.mole_fraction_dict().get('H2', 0)

We see the two methods take close to the same time to run.
Another option would be to access mole fractions from the mole-fraction array.

In [ ]:
%%timeit

gas.X[gas.species_index('H2')]

This is better!
What's more, the species index doesn't change, so if we store the index to a variable at the start of the run, we can
 just reuse the variable instead of re-calling the `species_index` method repeatedly.

In [ ]:
i_H2 = gas.species_index('H2')

In [ ]:
%%timeit

gas.X[i_H2]

Less than half the time of our original method!?

That could add up - we'll take it!

### 2f. Storing results in a dictionary

We've seen how we can use lists to track variables at each step of a simulation run.
However, even with the relatively few variables we tracked in the last run, we ended up with a lot of variables
 floating around for all those lists.

A cleaner way to organize results would be in a `dict`, with the names of the variables as the keys and `lists` as the
 stored values.

Since we'll want a new one of these dictionaries for every simulation run, we'll write a function to initialize one.

In [ ]:
def new_result_dict(sln: ct.Solution, species_list=None):
    # if the species list is None, make entries for all the species in the Solution object
    if species_list is None:
        species_list = sln.species_names
    else:  # remove any species in the list not in the Solution to prevent errors
        species_list = list(species_list)  # make sure the species_list is of type list
        for sp in species_list:
            if sp not in sln.species_names:
                print(f"{sp} not in mechanism; removed from tracked species.")
                species_list.remove(sp)

    # start initializing the dictionary with basic properties - time, temperature (T), pressure (P)
    result_dict = {'time':[],
                   'T':[],
                   'P':[]}
    # now add entries for the species
    for sp in species_list:
        result_dict[sp] = []
    # finally, add entries with a list of the species and their indices
    result_dict['species'] = species_list
    result_dict['indices'] = [sln.species_index(sp) for sp in species_list]

    return result_dict

In [ ]:
# a dictionary with default arguments
new_result_dict(gas)

In [ ]:
# a dictionary for a specified list of species
species_of_interest = ['H2', 'O2', 'H2O', 'OH', 'H', 'HO2']
results = new_result_dict(gas, species_of_interest)
results

We could now, also, define a function for extracting the state from a ReactorNet and Solution object at its current time
 step and append that data to the lists in our dictionary

In [ ]:
def add_to_result(sln, network, res_dict):
    res_dict['time'] += [network.time]
    res_dict['T'] += [sln.T]
    res_dict['P'] += [sln.P]
    for sp, x in zip(res_dict['species'], sln.X[res_dict['indices']]):
        res_dict[sp] += [x]

And we can check that it works as we expect:

In [ ]:
add_to_result(gas, rnet, results)
results

Now we're ready to run a simulation again, which should be a lot easier this time around.

Let's see what that looks like, starting again from the beginning:

In [ ]:
# setup the reactor, result objects
r1, rnet = setup_simulation(gas)
results = new_result_dict(gas, species_of_interest)

# run the simulation
t_stop = 1e-3
while rnet.time < t_stop:
    rnet.step()
    add_to_result(gas, rnet, results)

In [ ]:
# check that the results look like what we expect
plt.figure(figsize=(8, 4))
plt.semilogx(results['time'], results['T'])
plt.xlabel('Time (s)')
plt.ylabel('Temperature (K)')
plt.xlim((1e-6, t_stop))

plt.figure(figsize=(8, 6))
for sp in results['species']:
    plt.loglog(results['time'], results[sp], label=sp)
plt.legend()
plt.xlabel('Time (s)')
plt.ylabel('Mole Fractions (-)')
plt.xlim((1e-6, t_stop))
plt.ylim((1e-6, 1))

And they do!

At some time (the ignition delay time, IDT), fuel $H_2$ and oxygen $O_2$ are rapidly consumed, forming water $H_2O$
 as the primary product.
$OH$ is formed as a combustion intermediate, and persists after ignition as an equilibrium species at high temperatures.
Other radicals ($H$ and $HO_2$) reach peak concentrations at the time of ignition, settling to lower equilibrium
 concentrations at later times.

Since we'll often want to visualize results, let's wrap our plotting code into its own function:

In [ ]:
def plot_simulation_result(result_dict, logx=True, tmin=1e-6, tmax=None):
    ts = result_dict['time']
    if tmax is None:
        tmax = ts[-1]

    # plot temperature vs. time
    plt.figure(figsize=(8, 4))
    if logx:
        plt.semilogx(ts, result_dict['T'])
    else:
        plt.plot(ts, result_dict['T'])
    plt.xlabel('Time (s)')
    plt.ylabel('Temperature (K)')
    plt.xlim((tmin, tmax))

    # plot mole fractions vs. time
    plt.figure(figsize=(8, 6))
    for sp in result_dict['species']:
        if logx:
            plt.loglog(ts, result_dict[sp], label=sp)
        else:
            plt.semilogy(ts, result_dict[sp], label=sp)
    plt.legend()
    plt.xlabel('Time (s)')
    plt.ylabel('Mole Fractions (-)')
    plt.ylim((1e-6, 1))
    plt.xlim((tmin, tmax))

So now we can set up, run, and visualize the results of a simulation all in just a few lines of code:

In [ ]:
r1, rnet = setup_simulation(gas, T=1000)
results = new_result_dict(gas, species_of_interest)

# run the simulation
t_stop = 1e-3
while rnet.time < t_stop:
    rnet.step()
    add_to_result(gas, rnet, results)

# visualization using default arguments
plot_simulation_result(results)
# visualization with custom settings
plot_simulation_result(results, tmin=0, tmax=5e-4, logx=False)

Or, we could define a single function to do it all for us.
We'll get there in the next section...

## 3. Ignition Delay Times and Sensitivity Analysis

We've now seen how to get species and temperature time histories from a kinetic simulation.
As was introduced in the slides at the start of this problem session, we still want to be able to:
* Calculate/predict an ignition delay time $$\tau_{ign}$$ using a kinetic simulation
* Perform sensitivity analysis to determine to which reaction rates $$\tau_{ign}$$ is most sensitive

### 3a. Defining $\tau_\mathrm{ign}$:

There are a number of ways in which $\tau_{ign}$ can be defined and measured.
Experimentally, the ignition time may be measured from a rise in pressure, a rise in emission, or from a change in laser-absorption
 measurements.
These measurements can all produce slightly different results, making comparison between simulations and results
 challenging.

Similarly, a number of different values of $\tau_{ign}$ can be defined from simulation results.
For example, we could define ignition based on the peak rate of temperature increase, peak removal rate of fuel
 ($H_2$), peak production rate of products ($H_2O$), or the peak concentration of some radical species (e.g. $H$).
In a constant volume reactor, we could also use pressure, but our results have constant pressure by definition.

__Peak Radical Concentration:__

To start, we will consider the simplest definition, the peak concentration of the transient hydrogen radical.
From our time-series results, we saw that the mole fraction of atomic hydrogen peaks around the time of ignition,
 a feature that may make it a useful definition.

In order to extract the ignition time based on the maximum concentration, we can use the `argmax` function of the
 `numpy` package to get the index of the maximum value from an array.

_Note:_ While we could convert our list to an array before passing to `argmax`, this step is unnecessary, as that
 conversion is handled automatically by `numpy`.

In [ ]:
# get the index of the maximum mole fraction of H
iH_max = np.argmax(results['H'])

# find the time associated with that list index
t_ign_H = results['time'][iH_max]

print(f't_ign_H = {t_ign_H*1e6:4.1f} us')

In [ ]:
# plot the result to see we found the right point
plt.plot(results['time'], results['H'], '-k')
plt.plot(t_ign_H, results['H'][iH_max], '*r', ms=10)  # "ms" is the keyword argument for the "marker_size"
plt.xlim((0.8*t_ign_H, 1.2*t_ign_H))

__Peak Rate of Temperature Change:__

Another seemingly logical definition would be to consider ignition as the timing of the maximum rate of increase in
 the temperature.
To calculate the rate of change, we can again use `numpy`.
The `diff` function calculates the difference between consecutive array (or list) elements, allowing us to perform a
 numerical differentiation.

Since numerical differentiation will return one fewer element than we start with, we will also use `diff` to help
 calculate the mean time between time steps to match up with our rate of temperature change.

In [ ]:
dT = np.diff(results['T'])
dt = np.diff(results['time'])

dT_dt = dT / dt
t_mean = np.array(results['time'][:-1]) + (dt / 2)

idT_max = np.argmax(dT_dt)
t_ign_dT = t_mean[idT_max]

print(f't_ign_dT = {t_ign_dT*1e6:4.1f} us')

We find that the IDT calculated using the rate of temperature change is quite close to that found using the hydrogen
 radical, with a difference of only about 1%.

We can visualize this result as well; this time, we choose to do that a little differently:

In [ ]:
# visualize the result
#   first plot the raw temperature vs. time
plt.plot(results['time'], results['T'], '-k', label='T')
plt.ylabel('Temperature (K)')
plt.xlim((0.8*t_ign_H, 1.2*t_ign_H))
#   next we'll overlay a second y-axis and plot the dT/dt
plt.twinx()
plt.plot(t_mean, dT_dt, '-r', label='dT/dt')
plt.ylabel('dT/dt (K/s)')
#   the following lines just make the right axis red to match the data
plt.gca().spines['right'].set_color('r')
plt.gca().tick_params(axis='y', colors='r')
plt.gca().yaxis.label.set_color('red')
#   lastly, we'll add a vertical line marking the ignition time
plt.axvline(t_ign_dT, label='t_ign', c='b', ls=':')

We can use the same type of analysis as we did for temperature to look at ignition times based on fuel consumption and
 product formation.
We'll define some functions to make calculating IDTs easier to do repeatedly.

In [ ]:
def idt_x_max(result_dict, x_key, save_to_dict=True, print_out=False):
    x = result_dict[x_key]

    ix_max = np.argmax(x)  # flips the sign of dx_dt if keyword arg "sign" is negative
    t_ign_x = result_dict['time'][ix_max]

    if print_out:
        print(f'IDT_{x_key+"_max":10s} = {t_ign_x*1e6:4.1f} us')

    if save_to_dict:
        idt_dict = result_dict.get('idt', {})
        idt_dict[x_key] = t_ign_x
        result_dict['idt'] = idt_dict

def idt_dx_dt(result_dict, x_key, sign=1, save_to_dict=True, print_out=False):
    dx = np.diff(result_dict[x_key])
    dt = np.diff(result_dict['time'])

    dx_dt = dx / dt
    t_mean = np.array(result_dict['time'][:-1]) + (dt / 2)

    idx_max = np.argmax(np.sign(sign) * dx_dt)  # flips the sign of dx_dt if keyword arg "sign" is negative
    t_ign_dx = t_mean[idx_max]

    if print_out:
        print(f'IDT_d{x_key+"/dt":9s} = {t_ign_dx*1e6:4.1f} us')

    if save_to_dict:
        idt_dict = result_dict.get('idt', {})
        idt_dict[x_key] = t_ign_dx
        result_dict['idt'] = idt_dict

These two functions allow us to calculate any number of IDTs, adding them to the result dictionary:

In [ ]:
idt_x_max(results, 'H', print_out=True)
idt_dx_dt(results, 'T', print_out=True)
idt_dx_dt(results, 'H2', sign=-1, print_out=True)
idt_dx_dt(results, 'H2O', print_out=True)

results['idt']

And all these definitions can be bundled into a function to calculate all the IDTs for a given result:

In [ ]:
def calculate_idt(result_dict):
    idt_dx_dt(result_dict, 'T')
    idt_x_max(result_dict, 'H')
    idt_dx_dt(result_dict, 'H2', sign=-1)
    idt_dx_dt(result_dict, 'H2O')

In [ ]:
calculate_idt(results)
results['idt']

### 3b. Temperature Dependence of $\tau_\mathrm{ign}$

We may want to see how the ignition delay time varies with temperature.
To do this, we'll bundle everything required to run and analyze a simulation into a single function, then run this
 function repeatedly at different initial temperatures.

To start, let's define the function, which will accept and pass through many of the optional, keyword arguments
 we have been implementing along the way:

In [ ]:
T0 = 1200  # K
P0 = 101325  # Pa (1 atm)
phi = 1.
fuel = {'H2':1.}
oxidizer = {'AR':.79, 'O2':.21}  # "airgon"

def simulate_idt(sln, T=T0, P=P0, phi=phi, f=fuel, ox=oxidizer, species_of_interest=None, t_stop=1e-3):

    # setup the reactor, result objects
    r1, rnet = setup_simulation(gas, T=T, P=P, phi=phi, f=f, ox=ox)
    results = new_result_dict(gas, species_list=species_of_interest)

    # run the simulation
    while rnet.time < t_stop:
        rnet.step()
        add_to_result(gas, rnet, results)

    calculate_idt(results)

    return results['idt']

We can then run our IDT calculation function over a range of initial temperatures...

In [ ]:
Ts = np.arange(1100, 1801, 100)
print(Ts)

idt_list = []
for T in Ts:
    idt_list += [simulate_idt(gas, T=T)]

idt_list

And plot our results on an Arrhenius-type diagram

In [ ]:
for key in idt_list[0].keys():
    idts = [idt_dict[key] for idt_dict in idt_list]
    plt.semilogy(1000/Ts, idts, ':o', label=str(key))

plt.legend()
plt.xlabel('1000/T (1/K)')
plt.ylabel('IDT (s)')

From this diagram, we can make a few observations:
1. IDTs defined using temperature, fuel, and water closely agree
2. IDT defined using hydrogen radicals are systematically longer
3. The IDTs closely follow an Arrhenius trend

This last point suggests that the ignition delay time might be defined well be an Arrhenius-type expression, allowing
 the IDT (a global parameter) to be associated with a characteristic activation energy.
You will be asked to calculate such an activation energy as part of the problem set.

### 3c. $\tau_\mathrm{ign}$ Sensitivity Analysis

Now that we know how to calculate an IDT, we can look at how to calculate the sensitivity of the ignition delay time
 to changes in reaction rates.

If we were using Chemkin II (the old, free version we used to teach in ME362B), you would do this by manually
 editing the mechanism text file between each simulation -- not fun!

Fortunately, Cantera makes it much easier to change reaction rates by adjusting a multiplier assigned to each rate using
 the `set_multiplier` method.

To calculate the sensitivity, we perturb one reaction at a time by some small, fractional amount $f_k = dk_j / k_j$.
We then compare the IDTs between the base (unperturbed) mechanism and that with the rate changed.

We'll first calculate the sensitivity of IDT to the first reaction in the mechansim at T = 1200 K and P = 1 atm.

In [ ]:
# calculate the base result
T_sens = 1200
P_sens = 101325
IDT_def = 'T'  # this is the definition of IDT we'll use in our analysis

gas.set_multiplier(1.)  # this makes sure all the multipliers are set to 1.
IDT_base = simulate_idt(gas, T=T_sens, P=P_sens)[IDT_def]

# perturb the mechanism and calculate the new IDT
f_k = 0.05
i_rxn = 0
gas.set_multiplier((1+f_k), i_rxn)
IDT_sens = simulate_idt(gas, T=T_sens, P=P_sens)[IDT_def]

# calculate the IDT sensitivity
dIDT_IDT = (IDT_sens - IDT_base) / IDT_base
sensitivity = dIDT_IDT / f_k
print(f'[{i_rxn}] {gas.reaction_equation(i_rxn)}: Sensitivity = {sensitivity:4.3f}')

This tells us that increasing the rate of the first reaction in the mechanism results in a _decreasing_ the IDT
(i.e. _faster_ ignition).

We can now just wrap this calculation in a `for` loop over all the reactions in the mechanism.
Here, we'll store all the IDTs in an array as we go, then calculate our sensitivities at the end.

In [ ]:
# calculate the base result
T_sens = 1200
P_sens = 101325
IDT_def = 'T'  # this is the definition of IDT we'll use in our analysis

gas.set_multiplier(1.)  # this makes sure all the multipliers are set to 1.
IDT_base = simulate_idt(gas, T=T_sens, P=P_sens)[IDT_def]

# perturb the mechanism and calculate the new IDT
f_k = 0.05
IDT_sens = np.zeros(gas.n_reactions, dtype=float)

for i in range(gas.n_reactions):
    gas.set_multiplier(1)
    gas.set_multiplier((1+f_k), i)
    IDT_sens[i] = simulate_idt(gas, T=T_sens, P=P_sens)[IDT_def]

# calculate the IDT sensitivity
dIDT_IDT = (IDT_sens - IDT_base) / IDT_base
sensitivities = dIDT_IDT / f_k

print(sensitivities)

Which now gives us an array of sensitivities, indexed in order of the reactions in the mechanism.

In order to visualize these results, a horizontal bar plot is conventionally used.
We first sort the reactions and their equations by the sensitivity, using `np.argsort` to get a sorted list of indices.

In [ ]:
i_sort = np.argsort(sensitivities)[::-1]

sorted_sens = sensitivities[i_sort]
sorted_eqns = np.array(gas.reaction_equations(i_sort))

plt.figure(figsize=(8, 15))
plt.barh(sorted_eqns, sorted_sens)
plt.xlabel('IDT Sensitivity')
plt.axvline(0, ls=':', c='k')

At the top, we see a few reactions display negative sensitivies;
 increasing the rate of these reactions increases the rate at which ignition occurs.

Many of the reactions, however, display (small) positive sensitivities;
 increasing the rate of these reactions actually slows down the rate at which ignition occurs.

The problem set will ask you to calculate sensitivities for a different reactive system and comment on why some
 reactions promote (speed up) ignition while others inhibit it.

We may find it more helpful to only view a subset of the most sensitive reactions in performing an analysis.

One way to do this would be to only show sensitivities with a magnitude above a certain threshold using boolean
 indexing:

_Note:_ To use boolean indexing, our sensitivies and equantion names must be in the form of numpy arrays, **not** lists.
Lists do not support boolean indexing.

In [ ]:
threshold_sens = 0.01

above_thresh = np.abs(sorted_sens) > threshold_sens
above_thresh

In [ ]:
plt.figure(figsize=(8, 5))
plt.barh(sorted_eqns[above_thresh], sorted_sens[above_thresh])
plt.xlabel('IDT Sensitivity')
plt.axvline(0, ls=':', c='k')

Alternatively, we can use sorting to show only a certain number of the most sensitive reactions:

In [ ]:
n_max = 6

i_abs_sort = np.argsort(np.abs(sensitivities))[::-1]  # flip the order to sort from high to low
n_max_sens = sensitivities[i_abs_sort[:n_max]]
n_max_eqns = np.array(gas.reaction_equations(i_abs_sort[:n_max]))

i_sort_nmax = np.argsort(n_max_sens)[::-1]
n_sort_sens = n_max_sens[i_sort_nmax]
n_sort_eqns = n_max_eqns[i_sort_nmax]

In [ ]:
plt.figure(figsize=(8, 3))
plt.barh(n_sort_eqns, n_sort_sens)
plt.xlabel('IDT Sensitivity')
plt.axvline(0, ls=':', c='k')

In [ ]:
plt.figure(figsize=(8, 3))
plt.barh(range(len(n_sort_sens)), n_sort_sens)
plt.yticks(range(len(n_sort_sens)), n_sort_eqns)
plt.xlabel('IDT Sensitivity')
plt.axvline(0, ls=':', c='k')

The difference between the two plots above is how we handle passing y-values to the `barh` function.

In the first cell, we pass the array `n_sort_eqns`, and end up with one less bar than we were expecting.

In the cell immediately above, we instead pass `range(len(n_sort_sens))` as the y-values, then relabel the ticks with
 the `n_sort_eqns` array.

Doing this reveals that their is a duplicate reaction $H2 + O <=> H + OH$ in the mechanism.
Because y-values cannot be repeated, one is excluded in the first plot, where the strings define the y values.
In the second plot, sensitivities are ploted against numbers, that are only relabeled with strings, so they both appear.

### 3d. Temperature Dependence of IDT Sensitivity

In kinetics research, we often want to know how the relative importance of reactions changes with different
 conditions (e.g. temperature, pressure, equivalence ratio, oxygen mole fraction, etc.) or varies between different
 kinetic mechanisms.

Our last example in this notebook will demonstrate how you might look at how IDT sensitivity changes with temperature.
In your problem set, you will be asked to perform a similar analysis but while varying a different parameter.

As has been the case throughout this notebook, we'll define a function to do basically all our work for us.
In this case, at a given set of conditions, we'll have our function calculate all our sensitivities and optionally
 produce a plot of the most important reactions.

In [ ]:
# we redefine our base parameters for good measure
T0 = 1200  # K
P0 = 101325  # Pa (1 atm)
phi = 1.
fuel = {'H2':1.}
oxidizer = {'AR':.79, 'O2':.21}  # "airgon"

# then define our function, taking advantage of those we have already written
def calculate_sensitivities(sln, T=T0, P=P0, phi=phi, f=fuel, ox=oxidizer, t_stop=1e-3,
                            plot_result=True, n_sens_max=10, title=None, IDT_def='T'):
    # calculate the base result
    sln.set_multiplier(1.)  # this makes sure all the multipliers are set to 1.
    IDT_base = simulate_idt(sln, T=T, P=P, phi=phi, f=f, ox=ox, t_stop=t_stop)[IDT_def]

    # perturb the mechanism and calculate the new IDT
    f_k = 0.05
    IDT_sens = np.zeros(sln.n_reactions, dtype=float)

    for i in range(sln.n_reactions):
        sln.set_multiplier(1)
        sln.set_multiplier((1+f_k), i)
        IDT_sens[i] = simulate_idt(sln, T=T, P=P, phi=phi, f=f, ox=ox, t_stop=IDT_base*3)[IDT_def]

    # calculate the IDT sensitivity
    dIDT_IDT = (IDT_sens - IDT_base) / IDT_base
    sensitivities = dIDT_IDT / f_k

    if plot_result:
        i_abs_sort = np.argsort(np.abs(sensitivities))[::-1]  # flip the order to sort from high to low
        n_max_sens = sensitivities[i_abs_sort[:n_sens_max]]
        numbered_eqns = [f'{eqn} (r{i})' for i, eqn in enumerate(gas.reaction_equations())]
        n_max_eqns = np.array(numbered_eqns)[i_abs_sort[:n_sens_max]]

        i_sort_nmax = np.argsort(n_max_sens)[::-1]
        n_sort_sens = n_max_sens[i_sort_nmax]
        n_sort_eqns = n_max_eqns[i_sort_nmax]

        plt.figure(figsize=(8, n_sens_max/2))
        plt.barh(range(len(n_sort_sens)), n_sort_sens)
        plt.yticks(range(len(n_sort_sens)), n_sort_eqns)
        plt.xlabel('IDT Sensitivity')
        plt.axvline(0, ls=':', c='k')

        if title is not None:
            plt.title(title)

    return sensitivities

Let's try the new mega-function:

In [ ]:
calculate_sensitivities(gas, T=1500)

That's a lot of analysis packed into one line of code!

As a final step, let's wrap it into a loop and see how the most sensitive reactions change with temperature.

In [ ]:
Ts = [800, 850, 900, 1000, 1200, 1500, 2000]

for T in Ts:
    calculate_sensitivities(gas, T=T, title=f'T = {T:4.0f} K', t_stop=20, n_sens_max=6)